In [4]:
import time
import rlapy as rla
import numpy as np
import scipy.linalg as la
from notebooks.least_squares.helpers import ols_data


def time_alg(dims, spectrum, prop_range, alg, tol, seeds, lapack=False):
    n_rows, n_cols = dims
    lapack_times = np.zeros(seeds.size)
    rlapy_times = np.zeros(seeds.size)
    iter_lim = dims[1]
    for i, seed in enumerate(seeds):
        rng = np.random.default_rng(seed)
        A, b = ols_data(n_rows, n_cols, spectrum, prop_range, rng, only_Ab=True)
        if lapack:
            tic = time.time()
            la.lstsq(A, b)
            lapack_times[i] = time.time() - tic
        tic = time.time()
        alg.exec(A, b, tol, iter_lim, rng)
        rlapy_times[i] = time.time() - tic
        del A  # encourage Python to garbage collect
    return lapack_times, rlapy_times

In [5]:
main_rng = np.random.default_rng(897389723094)
num_runs = 2
run_lapack = False
seeds = main_rng.integers(0, int(1e10), size=num_runs)
dims = (100000, 2000)
kappa = 1e2
sap = rla.SAP1(rla.srct_operator, sampling_factor=5)

# linearly decaying spectrum
lin_spec = np.linspace(kappa**0.5, kappa**-0.5, num=dims[1])
llapackt, lrandlat = time_alg(dims, lin_spec, 0.25,
                              sap, 1e-10, seeds, run_lapack)
print(llapackt)
print(lrandlat)

# exponentially decaying spectrum
exp_spec = np.logspace(np.log10(kappa)/2, -np.log10(kappa)/2, num=dims[1])
elapackt, erandlat = time_alg(dims, exp_spec, 0.25,
                              sap, 1e-10, seeds, run_lapack)
print(elapackt)
print(erandlat)

[0. 0.]
[12.3502326 12.0530405]
[0. 0.]
[12.26121092 12.1584971 ]


In [6]:
run_lapack = False
sap.sketch_op_gen = rla.sjlt_operator

llapackt, lrandlat = time_alg(dims, lin_spec, 0.25,
                              sap, 1e-10, seeds, run_lapack)
print(llapackt)
print(lrandlat)

elapackt, erandlat = time_alg(dims, exp_spec, 0.25,
                              sap, 1e-10, seeds, run_lapack)
print(elapackt)
print(erandlat)


[0. 0.]
[7.90113378 7.92779541]
[0. 0.]
[8.21894622 8.06815386]
